In [1]:
import pandas as pd
import numpy as np
import re
import os
from os.path import join
import warnings
warnings.simplefilter("ignore")

deposit_dtype = {'채무자키':str, '입금키':str, '계좌키':str, '계좌번호':str, '입금고정키':str, '타채무자키':str}

# 출력 옵션
pd.options.display.float_format = '{:,.0f}'.format

### LM035(매각채권) - 그냥 수작업
- 매각이 여러건이면 읽을 파일이 많고, 단건이면 수작업도 금방 끝남

### LM036(매입채권 : 관리채권)

In [2]:
# 전체 원데이터 읽기

######################################################
path_raw_data = r"D:\3.자산\전산 dataset\솔림\240930\pkl"
fn_raw_data = "2.솔림_전체_원데이터_240930_by새채무자키"
######################################################

raw_data = pd.read_pickle(join(path_raw_data, fn_raw_data + ".pkl"))

# 종료, 종료요청 제외
raw_data = raw_data.query('~새채무상태.str.contains("종료")')
raw_data.reset_index(inplace=True, drop=True)
print(f'채권수 : {len(raw_data):,} / 반영OPB합 : {raw_data.반영OPB.sum():,}')

# 원채권사, 매각사 코드 merge
# 채무자조회파일에서 매각사 불러오기
file_list = os.listdir(path_raw_data)
fn_debt = [file for file in file_list if file.startswith("채무자조회")][0]

debt = pd.read_pickle(join(path_raw_data, fn_debt))[["채무자키", "매각사"]]

# 코드 파일 불러오기
원채권사코드 = pd.read_excel(r"D:\3.자산\금감원\기관코드표.xlsx", sheet_name="원채권사", dtype={"원채권사코드":str})
매각사코드 = pd.read_excel(r"D:\3.자산\금감원\기관코드표.xlsx", sheet_name="매각사", dtype={"매각사코드":str})

# 병합
raw_data = raw_data.merge(debt, on="채무자키", how="left")
raw_data = raw_data.merge(원채권사코드, on="원채권사", how="left")
raw_data = raw_data.merge(매각사코드, on="매각사", how="left")

# 법조치 읽기
fn_event = [file for file in file_list if file.startswith("법조치조회")][0]
event = pd.read_pickle(join(path_raw_data, fn_event))[["채무자키", "법조치구분", "접수일"]]
len(event)


# 새채무자키 merge
event = pd.merge(event, raw_data.drop_duplicates('채무자키')[["채무자키","새채무자키"]] , on="채무자키", how='left')

# 법조치 필터링(집행권원만 할거면 자산정리_통합파일에서, **신청일 살려야)
# 새채무자키 없는 거 삭제(종결건 삭제)
event.query('~새채무자키.isna()')
# 접수일 없는 거 삭제
event = event.query('접수일 != ""')
# 법조치구분 선택
comp = r"가?압류|가?처분|강제|경매|집행|담보|보전|본안|사해행위|시효연장|유체|재산관계|재산명시|전자독촉|지급명령|불이행|추심"
event = event.query('법조치구분.str.contains(@comp)')

# 접수일 역정렬
event.sort_values("접수일", ascending=False, inplace =True)
event.drop_duplicates(subset='새채무자키', keep='first', inplace=True)

# raw_data에 열추가(merge)
raw_data = raw_data.merge(event[["새채무자키", "접수일"]], on="새채무자키", how="left").fillna("")

# 병합잘못되어 계좌키 중복건 있는지 체크
raw_data[raw_data.계좌키.duplicated()]

채권수 : 28,478 / 반영OPB합 : 373,742,676,265


,채무자키,계좌키,타채무자키,채권구분,채권상태,새채무상태,채무상태,담당자,새보증인상태,반영OPB,...,주소,새채무자키,보증인성명인,보증인상태,추심가능여부,새보증인상태by새채무자키,매각사,원채권사코드,매각사코드,접수일


In [3]:
# 복붙용 파일 만들기
lm036 = pd.DataFrame()

lm036["고객번호"] = raw_data.채무자키
lm036["계약번호"] = raw_data.계좌번호
lm036["차주명"] = raw_data.성명
lm036["차주실명번호"] = raw_data.주민등록번호.apply(lambda x: re.sub("-","", x)+"000" if len(re.sub("-","", x)) == 10 else re.sub("-","", x))
lm036["차주구분"] = np.where(raw_data.개인법인=="법인", "03", np.where(raw_data.주민등록번호.str[7].astype(int) % 2 == 1, "01", "02"))
lm036["매입회차"] = ""
lm036["매입일"] = raw_data.매입일
lm036["대출일"] = raw_data.최초대출일
lm036["대출종류"] = "01"
lm036["상환금액"] = raw_data.회수합계
lm036["만기일"] = raw_data.대출만기일
lm036["최초계약금액"] = raw_data.은행대출원금
lm036["매입서류상잔액"] = raw_data.매입OPB # 2406이후 매입시 반영OPB로(매입OPB컬럼은 7월 자산정리파일부터 있음), raw_data.최초원금 + raw_data.최초법비용 # raw_data의 최초금액은 매입최초금액
lm036["실제매입금액"] = ""
lm036["기준일장부가액"] = "" # 재무제표 오재홍팀장님 작성건 2406부터 반영opb로, raw_data.현재원금 + raw_data.현재법비용
lm036["금리"] = raw_data.연체이자율
lm036["최종수령일"] = raw_data.최종거래일
lm036["이자연체일수"] = 0
lm036["만기연체일수"] = 0
lm036["시효완성일"] = raw_data.시효완성일
lm036["지급명령신청일"] = raw_data.접수일
lm036["채권서류존재유무"] = np.where(raw_data.원인서류유무==0,"N", "Y")

x = raw_data.새채무상태
y = raw_data.채무상태
z = raw_data.분납연체일
conds = [x.str.contains("개인회생"), x.str.contains("신용회복"), y.str.contains("사망"), ~z.isna()]
values = ["03", "04", "07", "05"]
lm036["현상태"] = np.select(conds, values, default="02")
lm036["원인채권회사번호"] = raw_data.원채권사코드
lm036["원인채권회사명"] = raw_data.원채권사
lm036["채권양도회사번호"] = raw_data.매각사코드
lm036["양도금융회사명"] = raw_data.매각사
lm036["채권추심위임일"] = ""
lm036["채권추심위임기관"] = ""
lm036["연대보증여부"] = "N"

In [4]:
# 엑셀로 보내기
lm036.to_excel(r"D:\3.자산\금감원\lm036_새채무자키.xlsx", index=False)

# 신용회복 원상회복 때문에 회수내역은 입금 다운 받아서 재작성해야함

### 회수내역
- 매입일 가장 빠른 날짜부터 기준일까지, 계좌키표시, 만건씩 귾어서, 기본값으로 조회

In [2]:
#################################################
path_dir_입금 = r"D:\3.자산\금감원\2024 전반기\참고"
path_fn_입금 = "대성_입금조회새창_20240717_1045.xlsx"
기준일 = "2024-06-30"
#################################################

# 입금 파일 읽고 회수내역 재작성 (신복 원상회복 때문에..)
입금 = pd.read_excel(join(path_dir_입금, path_fn_입금), dtype=deposit_dtype)

In [3]:
deposit = 입금[["채무자키", "계좌키", "입금합계","입금일", "입금구분", "입금기간"]].copy()

# 불필요 행 삭제 
cond1 = deposit.입금일 <= 기준일
cond2 = deposit.입금구분.str.contains("약정분납|임의변제|개인회생|신용회복|채권압류|배당급|유체동산|일반입금|대위변제|부동산경매|내사입금|기타")
deposit['입금기간'] = deposit['입금기간'].astype(str)
cond3 = ~(deposit.입금기간.str.contains("환불|회수제외|매입전|매각후"))

deposit = deposit[cond1 & cond2 & cond3]

# 계좌별로 그룹화하기
계좌별회수내역 = deposit.groupby('계좌키')["입금합계"].sum().reset_index()

# 계좌번호 추가하기
계좌별회수내역 = 계좌별회수내역.merge(raw_data[["계좌키", "채무자키","계좌번호"]], on='계좌키')

# 계좌번호가 pk인지 확인(false만 나와야)
print("계좌번호 pk인지 확인(false만 나와야)")
print(계좌별회수내역.duplicated('계좌번호', keep=False).value_counts())

NameError: name 'raw_data' is not defined

In [8]:
계좌별회수내역.to_excel(join(path_dir_입금, "매입후_계좌별회수내역.xlsx"), index=False)
# 채무자키 + 계좌번호가 pk 임

#### 반기보고서용 회수내역 GC035

In [9]:
### 반기보고서용 회수내역 (당반기)
deposit = 입금[["채무자키", "계좌키", "입금합계","입금일", "입금구분", "입금기간"]].copy()

# 불필요 행 삭제 
cond1 = (deposit.입금일 <= 기준일) & ( "2024-01-01" <= deposit.입금일) ###############################
cond2 = deposit.입금구분.str.contains("약정분납|임의변제|개인회생|신용회복|채권압류|배당급|유체동산|일반입금|대위변제|부동산경매|내사입금|기타")
deposit['입금기간'] = deposit['입금기간'].astype(str)
cond3 = ~(deposit.입금기간.str.contains("환불|회수제외|매입전|매각후"))
deposit = deposit[cond1 & cond2 & cond3]

# 개인법인 merge
deposit = deposit.merge(raw_data[["계좌키", "개인법인"]], on='계좌키')

# 개인법인 회수내역 pivot
deposit.pivot_table(values='입금합계', index='개인법인', aggfunc='sum')

,입금합계
개인법인,
개인,159628742
개인사업자,4206
법인,11262666
